In [1]:
# print(model.model.transformer_module.decoder)
# print(model.class_predictor)
# - Get the linear layer with the class predictor 


import torch 
from datasets import concatenate_datasets, load_dataset
from torch.utils.data import Dataset 
from datasets import load_dataset
import cv2
from huggingface_hub import hf_hub_download
import json
from PIL import Image
import numpy as np
import os 
import torchvision as torchvision
from torch.utils.data import Dataset 
import torchvision.transforms as transforms
import torchvision as torchvision
import albumentations as A
import albumentations as A
from albumentations.pytorch import ToTensorV2

/home/chia/anaconda3/envs/bizon_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
repo_id = f"chloechia/loveda"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id, filename, repo_type="dataset"), "r"))
id2label = {int(k):v for k,v in id2label.items()}
print(id2label)

{0: 'Ignore', 1: 'Background', 2: 'Building', 3: 'Road', 4: 'Water', 5: 'Barren', 6: 'Forest', 7: 'Agricultural'}


# Dataloading

In [3]:
dataset = load_dataset("chloechia/correctloveda")
train_image = load_dataset("chloechia/correctloveda", split="train")
validation_image = load_dataset("chloechia/correctloveda", split="validation")
mask_dataset = load_dataset("chloechia/maskloveda")

Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--correctloveda-701d9bc57e72e64b/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100%|██████████| 2/2 [00:00<00:00,  9.70it/s]
Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--correctloveda-701d9bc57e72e64b/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--correctloveda-701d9bc57e72e64b/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--maskloveda-30d1338528c17a6a/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
100%|██████████| 2/2 [00:00<00:00, 73.52it/s]


In [4]:
train_image = train_image.remove_columns("label")
validation_image = validation_image.remove_columns("label")
train_mask_dataset = load_dataset("chloechia/maskloveda", split="train")
train_mask_dataset = train_mask_dataset.remove_columns("label")
val_mask_dataset = load_dataset("chloechia/maskloveda", split="validation")

Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--maskloveda-30d1338528c17a6a/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--maskloveda-30d1338528c17a6a/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [5]:
val_mask_dataset = load_dataset("chloechia/maskloveda", split="validation")
val_mask_dataset = val_mask_dataset.remove_columns("label")
val_mask_dataset = val_mask_dataset.rename_column("image", "mask")

Found cached dataset imagefolder (/home/chia/.cache/huggingface/datasets/chloechia___imagefolder/chloechia--maskloveda-30d1338528c17a6a/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [6]:
train_mask_dataset = train_mask_dataset.rename_column("image", "mask")

In [7]:
from datasets import concatenate_datasets, load_dataset

In [8]:
official_train = concatenate_datasets([train_image, train_mask_dataset], axis=1)
official_val = concatenate_datasets([validation_image, val_mask_dataset], axis=1)
official_train, official_val

(Dataset({
     features: ['image', 'mask'],
     num_rows: 1366
 }),
 Dataset({
     features: ['image', 'mask'],
     num_rows: 1669
 }))

In [9]:
masky = official_train[0]['image']
image_shape = masky.size

# Image Segmentation Class

In [10]:
class NormalizeTransform(object):
    def __call__(self, img):
        img = img / 255.0
        return img

In [11]:
import torchvision.transforms as transforms

# train_transform expects an img. train_transform(img)
train_transform = transforms.Compose([
    transforms.RandomRotation(90),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    NormalizeTransform()
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
    
    
])

mask_transform = transforms.Compose([
    transforms.RandomRotation(90),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    NormalizeTransform()
    # transforms.Normalize(mean=[0.5], std=[0.5])
    
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    NormalizeTransform()
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
    
    
    
])

mask_test_transform = transforms.Compose([
    transforms.ToTensor(),
    NormalizeTransform()
    # transforms.Normalize(mean=[0.5], std=[0.5])
    
])

# train_transform.transforms = list(train_transform.transforms)
# mask_transform.transforms = list(mask_transform.transforms)
# test_transform.transforms = list(test_transform.transforms)
# mask_test_transform.transforms = list(mask_test_transform.transforms)

# train_transform.transforms.append(transforms.Lambda(lambda x: x * 0.5 + 0.5))
# mask_transform.transforms.append(transforms.Lambda(lambda x: x * 0.5 + 0.5))
# test_transform.transforms.append(transforms.Lambda(lambda x: x * 0.5 + 0.5))
# mask_test_transform.transforms.append(transforms.Lambda(lambda x: x * 0.5 + 0.5))


In [14]:
from datasets import Dataset

class ImageSegmentationDataset(Dataset):
    """Image segmentation dataset."""

    def __init__(self, dataset, transform, mask_transform):
        """
        Args:
            dataset
        """
        self.dataset = dataset
        self.transform = transform
        self.mask_transform = mask_transform 
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        original_image = self.dataset[idx]['image']
        original_segmentation_map = self.dataset[idx]['mask']
        image = self.transform(original_image)
        mask = self.mask_transform(original_segmentation_map)


        return image, mask, original_image, original_segmentation_map


train_dataset = ImageSegmentationDataset(dataset=official_train, transform=train_transform, mask_transform=mask_transform)
test_dataset = ImageSegmentationDataset(dataset=official_val, transform=test_transform, mask_transform=mask_test_transform)

In [15]:
train_dataset[0][0].shape

torch.Size([3, 1024, 1024])

In [16]:
def color_palette():
    return [[255, 255, 255], [255,0,0], [255,255,0], [0,0,255], [159,129,183], [0,255,0], [255,195, 128]]

palette = color_palette()
palette

[[255, 255, 255],
 [255, 0, 0],
 [255, 255, 0],
 [0, 0, 255],
 [159, 129, 183],
 [0, 255, 0],
 [255, 195, 128]]

In [17]:
from transformers import Mask2FormerImageProcessor, Mask2FormerConfig, Mask2FormerForUniversalSegmentation, AutoImageProcessor

In [18]:
from PIL import Image
import cv2 as cv2 
#processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-small-cityscapes-semantic")
#model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-small-cityscapes-semantic")

CONFIG = "facebook/mask2former-swin-large-cityscapes-semantic"
processor = Mask2FormerImageProcessor.from_pretrained(CONFIG)
model = Mask2FormerForUniversalSegmentation.from_pretrained(CONFIG)

## Change the last Fully Connected Layer 

In [21]:
from torch import nn 
fc = nn.Linear(in_features=256, out_features=7)
model.class_predictor = fc
model.class_predictor

Linear(in_features=256, out_features=7, bias=True)

In [20]:
train_dataloader.dataset

NameError: name 'train_dataloader' is not defined

## Finetuning

In [22]:
import evaluate

metric = evaluate.load("mean_iou")

In [27]:
train_dataloader.dataset

AttributeError: 'ImageSegmentationDataset' object has no attribute '_info'

In [24]:
from torch.utils.data import DataLoader


In [28]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    # this function pads the inputs to the same size,
    # and creates a pixel mask
    # actually padding isn't required here since we are cropping
    batch = processor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_image"] = inputs[2]
    batch["original_segmentation_map"] = inputs[3]
    
    return batch

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [29]:
next(iter(train_dataloader))

TypeError: Unexpected type <class 'list'>

In [178]:
for i, (images, labels) in enumerate(train_dataloader):
    print(type(images))

TypeError: Unexpected type <class 'list'>

In [174]:
import torch

batch = next(iter(train_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k,v.shape)
  else:
    print(k,v[0].shape)

TypeError: Unexpected type <class 'list'>

In [117]:
import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

running_loss = 0.0
num_samples = 0
for epoch in range(20):
  print("Epoch:", epoch)
  model.train()
  for idx, batch in enumerate(tqdm(train_dataloader)): # this iterates through all the batches. one for loop = one batch is put through the training loop. train_dataloader is loading in the batches
      # Reset the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(
          pixel_values=batch["pixel_values"].to(device),
          mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
          class_labels=[labels.to(device) for labels in batch["class_labels"]],
      )

      # Backward propagation
      loss = outputs.loss
      loss.backward()

      batch_size = batch["pixel_values"].size(0)
      running_loss += loss.item()
      num_samples += batch_size

      if idx % 100 == 0:
        print("Loss:", running_loss/num_samples)

      # Optimization
      optimizer.step()

  model.eval()
  for idx, batch in enumerate(tqdm(test_dataloader)):
    if idx > 5:
      break

    pixel_values = batch["pixel_values"]
    
    # Forward pass
    with torch.no_grad():
      outputs = model(pixel_values=pixel_values.to(device))

    # get original images
    original_images = batch["original_images"]
    target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
    # predict segmentation maps
    predicted_segmentation_maps = processor.post_process_semantic_segmentation(outputs,
                                                                                  target_sizes=target_sizes)

    # get ground truth segmentation maps
    ground_truth_segmentation_maps = batch["original_segmentation_maps"]

    metric.add_batch(references=ground_truth_segmentation_maps, predictions=predicted_segmentation_maps)
  
  # NOTE this metric outputs a dict that also includes the mIoU per category as keys
  # so if you're interested, feel free to prcint them as well
  print("Mean IoU:", metric.compute(num_labels = len(id2label), ignore_index = 0)['mean_iou'])


# torch.save(model.state_dict(), 'model.pt')
# # with model.eval(), you're not updating the weights. 
# # {
#     'layer1.weight': tensor([...]),    # Example weight tensor of 'layer1'
#     'layer1.bias': tensor([...]),      # Example bias tensor of 'layer1'
#     'layer2.weight': tensor([...]),    # Example weight tensor of 'layer2'
#     'layer2.bias': tensor([...]),      # Example bias tensor of 'layer2'
#     ...
# }
# Each parameter in the model has a corresponding entry in the state dictionary. In the example above, 'layer1.weight' corresponds to the weight tensor of the first layer, 'layer1.bias' corresponds to the bias tensor of the first layer, and so on. The ellipsis ([...]) represents the values of the tensors.

# When you load the model.pt file using model.load_state_dict(torch.load('model.pt')), the state dictionary is loaded into the model, and the model's parameters are updated with the saved tensor values, restoring the model to the state it was in when it was saved.





Epoch: 0


  0%|          | 0/683 [00:00<?, ?it/s]


TypeError: Unexpected type <class 'list'>

In [130]:
for idx, batch in enumerate(train_dataloader):
    print(batch) # gives you the raw pixel values of the image (but some values are negative) 
    print(batch["mask_labels"]) # mask labels and class labels are different? 
    print(batch["class_labels"])
    break

ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-2.1179039478302, 2.640000104904175] which cannot be converted to uint8.

# Color Palette


In [100]:
from typing import Dict, Tuple

# building, road, barren, vegetation 
newidtocolor: Dict[int, Tuple[int, int, int]] = {
    "0": (70, 70, 70),
    "1": (128, 64, 128),
    "2": (152, 251, 152),
    "3":(107, 142, 35),
}

newidtocolor: Dict[int, Tuple[int, int, int]] = {
    int(key): value for key, value in newidtocolor.items()
}

newidtocolor

{0: (70, 70, 70), 1: (128, 64, 128), 2: (152, 251, 152), 3: (107, 142, 35)}